# Predicting Hourly Taxi Demand

## Summary

In this report, we will make hourly prediction of taxi demand in the Bronx, and determine the optimal number of taxis required accordingly. The data at hand is the historical trip data from taxis, combined with hourly weather data. The potential challenges and oppertunites of this analysis include:

* data quality: 
  
  the data need to be cleaned and processed to remove bias arising from technical errors and outliers of data.

* data smoothing:

  the original time series of hourly demand can be noisy for trend and seasonality recognition. The occational zero value of hourly demand around mid-night can also be problematic for the analysis. Therefore, data smoothing is necessary prior to modeling.

* seasonality:

  The taxi demand is subject to, but not limited to, the followings seasonalities: 
  * hour of the day: the taxi demand would surge during rush-hours.
  * day of the week: Friday night, Saturday and Sunday are more likely to yield higher demands than workdays. 
  * week of the month: The bi-weekly payroll days in United States are typically the first and 15/16th days of each month according to Forbes. Thus the first and third week can potentially have surging demand. However, this hypothesis need to be tested with the data. 
  * public holidays: We can encode all public holidays in the data in 2023 and test its importance in the prediction models.
  
  Note that to consider the potential impact of multiple seasonalities, we need to perform transformation to decompose the taxi demands time-series to its trend, seasonality and residuals.

* spatial factors:

  The spatial patterns in the taxi trip records may be helpful for taxi demand prediction. Specifcally, a surging number of drop-offs in the same borough may indicate that there is an event, and therefore impact the future taxi demand hours later. For example, if there are lots of drop-offs in Bronx or a neighoring borough at 9pm, it could be the cause of a concert at Bronx and it will lead to taxi demand surge in Bronx hours after the event. Therefore we need to encode the spatial factors of the drop-off locations and examine the impact of such features in our prediction models.

* Crowd effects:

  Similar to the spatial information, the abnormalty of the number of passengers in the taxi may indicate fluctutation of taxi demands hours later. e.g. group-trip for concerts. Therefore, we need to encode this information for predictions.

* Weather:

  we will incoporate the weather data into the prediction models. 
  
* prediction models:

  * Baseline Model:

     We will be using weighted moving average and exponential weighted moving average as the baseline models for hourly demand prediction. We will be using mean absolute percentage error (MAPE) for the performance evaluation. The rationals and details will be explained in the methods section below.

  * Improved Models:
     
      We will be consder model improvement by testing the following models below. Details will be in the method section.
     * smoothing approach: triple exponential moving average (TEMA)
     * machine learning approach: random forest + Fourier/TEMA features 
     * machine learning approach: weighted subspace random forest + Fourier/TEMA features
     * machine learning approach: xgboost + Fourier/TEMA features
    
  * Best Model:
     
     Details explaining the steps taken to enhance the model and the final results and comparison are shown in the methods section.

* Forecasting the First Week of September 2024:
     
     Using your best model, forecast the hourly demand for taxis in the Bronx for the first week of September 2024. Explain any additional steps or assumptions you took into consideration for this forecasting task, such as seasonal trends, external factors, or potential anomalies. 

* Optimal Number of Taxis:
   
     Determine the optimal number of taxis required to meet the predicted hourly demand in the Bronx. Make reasonable assumptions about factors such as average trip duration, taxi availability, and operational constraints. You might need to consider potential costs and benefits in your analysis. Explain your assumptions and the methodology used to calculate the optimal number of taxis. Discuss the implications of your findings on operational efficiency and customer satisfaction.








## Methods



### Data Quality Control

We first conduct data quality control before analysis. This includes checking:

* Data summary

  The original TaxiTrip data contains 66,000 rows or records. The columns include: tpep_pickup_datetime, tpep_dropoff_datetime,  passenger_count,  trip_distance, 
  PULocationID, and DOLocationID.  

* duplications

  662 duplicated rows are removed from the data. This is probably due to techinical errors. 

* Missing values

  5752 passenger_count entries and 38 trip_distance entries are missing values. we replace the missing value with the median value of the column with same pick-up and drop-off locationsIDs.

* Zero values

  13,369 trip-distances are zero. 2,685 records are with exact the same pick-up and drop-off date-time. We remove these 2,685 rows. For the rest of zero entries, we replace the zero trip-distances values with the median value of the subset trips data with the same pick-up and drop-off locations. Note that the high number of zero values implies that the original recorded trip distances may not be a reliable measurement of the trip information.

  294 passenger_count have zero values. Assuming this ride-hailing company did not mix delivery service records with ride-hailing records, we will replace the 0 values with the median non-zero value.
  
* Outlier values

  We examine the summary statistics of each numerical column and find outliers in passenger_count and trip_distance.
  There are 6 outliers in passenger_count values. i.e. we have 6 records of which each contains 11 passengers in a taxi. we replace these passenger_count with the median value of the column.
  
  There are also extremely outlier values of trip_distances (See Figure 1). The New York City has an area of 300.5 square-miles, therefore the theoretical maximum distances of a single drop-off in the NYC area should be sqrt(2)*sqrt(300.5) = 24.51 miles. However, consider that (i) there could be multiple drop-offs of multiple passengers in a single trip, and (ii) there were drop-offs outside of NYC, then the actual trip distances could be exceeding that limits. Therefore, instead we calculate the trip durations of each trip record, and calculate the expected trip_distances with the general speed limit of NYC, i.e. 25 mph. Specifically, for each suspicious outlier trip_distances (> 24.51 mile), we replace the recorded trip distance value with the lesser of these two values: (i) 25 mph times the recorded trip_durations for this trip and (ii) the original record trip_distance value. 

* trip durations
  
  Note we calculate the trip_durations from the pick-up date-time and drop-off date-time. There were 13 rows with pickup time later than drop-off time. Without proper hypothesis we will remove these 13 rows. There were also extremely long trip_durations (e.g. 3556 minutes = 59 hours). According to NYC government, "Both taxi and FHV drivers are prohibited from transporting passengers for more than 10 hours in any 24-hour period". Therefore we will remove all records that with trip_duration exceeding that limits. 
  
* Summary Statistics

  * At least 75% of trips have only one passenger.
  * The median trip distance is 4.9 miles. 75% of trips have trip distance < 10.1 miles.
  * The median trip duration is 27 minutes. 75% of trips have trip duration <= 43 minutes. Note that we will use this information later for calculating the optimal number of taxi deployment.
  


  


In [108]:
#libraries
import dask.dataframe as dd
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import scipy
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings

#load trip data
trip_data = pd.read_csv('/Users/laijiang/Documents/Pers/datatest/Data/dat/TaxiTrips_BronxOrigin2023.csv')

#print number of rows 
print('Number of rows in trip data: ', trip_data.shape[0])

#print the first few rows
print(trip_data.head())

#check if any rows are duplicated in trip_data
print('Number of duplicate rows in trip data: ', trip_data.duplicated().sum())


#QC 1: drop the dupliated records
trip_data = trip_data.drop_duplicates()
print('Number of rows in trip data: ', trip_data.shape[0])

#QC 2: missing values
print('Number of missing values in trip data: ', trip_data.isnull().sum())

#for the missing values in the column "passenger_count" and "trip_distance", we impute the missing values with the median of the column with same pick-up and drop-off locationsIDs
trip_data['passenger_count'] = trip_data['passenger_count'].fillna(trip_data.groupby(['PULocationID', 'DOLocationID'])['passenger_count'].transform('median'))
trip_data['trip_distance'] = trip_data['trip_distance'].fillna(trip_data.groupby(['PULocationID', 'DOLocationID'])['trip_distance'].transform('median'))

# QC 3: zero values
#check number of rows with trip_distance = 0
print('Number of rows with trip_distance = 0: ', trip_data[trip_data['trip_distance'] == 0].shape[0])

#number of rows with exactly the same value of columns tpep_pickup_datetime and tpep_dropoff_datetime
print('Number of rows with exactly the same value of columns tpep_pickup_datetime and tpep_dropoff_datetime: ', trip_data[trip_data['tpep_pickup_datetime'] == trip_data['tpep_dropoff_datetime']].shape[0])

#remove the rows with exactly the same value of columns tpep_pickup_datetime and tpep_dropoff_datetime. probably due to technical error
trip_data = trip_data[trip_data['tpep_pickup_datetime'] != trip_data['tpep_dropoff_datetime']]

#for the rest of zero values in the column "trip_distance", we impute the zero values with median value of the subset trips data with the same pick-up and drop-off locations. 
trip_data['trip_distance'] = trip_data['trip_distance'].replace(0, np.nan)
trip_data['trip_distance'] = trip_data['trip_distance'].fillna(trip_data.groupby(['PULocationID', 'DOLocationID'])['trip_distance'].transform('median'))

#check number of rows with passenger_count = 0
print('Number of rows with passenger_count = 0: ', trip_data[trip_data['passenger_count'] == 0].shape[0])

#replace the zero values in the column "passenger_count" with the median of the column
trip_data['passenger_count'] = trip_data['passenger_count'].replace(0, np.nan)
trip_data['passenger_count'] = trip_data['passenger_count'].fillna(trip_data['passenger_count'].median())


#QC 4: check the outliers of each column
print('Summary statistics of trip data: ', trip_data.describe())

#print the table of passenger_count counts
print('Counts of passenger_count in trip data: ', trip_data['passenger_count'].value_counts())

#replace the passenger_count values greater than 6 with the median of the column
trip_data['passenger_count'] = trip_data['passenger_count'].replace(11, np.nan)
trip_data['passenger_count'] = trip_data['passenger_count'].fillna(trip_data['passenger_count'].median())


#for these outliers of "trip_distance" > 24.51: 

#find calculate the trip duration 
def convert_to_unix(s):
    return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M").timetuple())


def fun_trip_durations(trp_dat):
    duration = trp_dat[['tpep_pickup_datetime','tpep_dropoff_datetime']]
    # pickups and dropoffs to unix time
    duration_pickup = [convert_to_unix(x) for x in duration['tpep_pickup_datetime'].values]
    duration_drop = [convert_to_unix(x) for x in duration['tpep_dropoff_datetime'].values]
    # calculate duration of trips
    durations = (np.array(duration_drop) - np.array(duration_pickup))/60  # in minutes
    # append durations of trips to a new dataframe
    new_frame = trp_dat
    new_frame['trip_durations'] = durations
    # average expectation (25 mph) of trip_distances = durations * 25
    new_frame['trip_distance_exp'] = new_frame['trip_durations'] * 25 / 60

   #the rows with trip_distance > 24.51 are replaced with the values of trip_distance_exp
    new_frame.loc[new_frame['trip_distance'] > 24.51, 'trip_distance'] = new_frame['trip_distance_exp']
    return new_frame



trip_dur_dat = fun_trip_durations(trip_data)
print(trip_dur_dat.head())

print('Summary statistics of trip data: ', trip_dur_dat.describe())

#Number of rows with negative trip_durations
print('Number of rows with negative trip_durations: ', trip_dur_dat[trip_dur_dat['trip_durations'] < 0].shape[0])
#show these rows with negative trip_durations
print(trip_dur_dat[trip_dur_dat['trip_durations'] < 0])

#remove the rows with negative trip_durations
trip_dur_dat = trip_dur_dat[trip_dur_dat['trip_durations'] > 0]
#remove the rows with trip_durations > 10*60 
trip_dur_dat = trip_dur_dat[trip_dur_dat['trip_durations'] < 10*60]


print('Summary statistics of trip data: ', trip_dur_dat.describe())


#Q-Q plot for checking iftrip_distance is normal and save figure
#add the diagonal line also
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.kdeplot(trip_dur_dat['trip_distance'].values, fill=True, color="b")
plt.title('PDF of trip_distance')
plt.subplot(1, 2, 2)
res = scipy.stats.probplot(trip_dur_dat['trip_distance'].values, plot=plt)
plt.plot([0,25],[0,25], color='r')
plt.savefig('trip_distance.png')
plt.close()

#Q-Q plot for checking iftrip_distance is log-normal and save figure
#add the diagonal line also
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.kdeplot(np.log(trip_dur_dat['trip_distance'].values), fill=True, color="b")
plt.title('PDF of log(trip_distance)')
plt.subplot(1, 2, 2)
res = scipy.stats.probplot(np.log(trip_dur_dat['trip_distance'].values), plot=plt)
plt.plot([0,10],[0,10], color='r')
plt.savefig('log_trip_distance.png')
plt.close()


#Q-Q plot for checking if trip-durations is normal ans save figure
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.kdeplot(trip_dur_dat['trip_durations'].values, fill=True, color="b")
plt.title('PDF of trip_durations')
plt.subplot(1, 2, 2)
res = scipy.stats.probplot(trip_dur_dat['trip_durations'].values, plot=plt)
plt.savefig('trip_duration.png')
plt.close()



#Q-Q plot for checking if trip-durations is log-normal ans save figure
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.kdeplot(np.log(trip_dur_dat['trip_durations'].values), fill=True, color="b")
plt.title('PDF of log(trip_duration)')
plt.subplot(1, 2, 2)
res = scipy.stats.probplot(np.log(trip_dur_dat['trip_durations'].values), plot=plt)
plt.savefig('log_trip_duration.png')
plt.close()



Number of rows in trip data:  66000
  tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0      2023-01-01 0:08       2023-01-01 0:41              1.0          25.84   
1      2023-01-01 0:27       2023-01-01 0:32              1.0           1.03   
2      2023-01-01 0:16       2023-01-01 0:16              4.0           0.00   
3      2023-01-01 0:00       2023-01-01 0:26              1.0           0.00   
4      2023-01-01 1:51       2023-01-01 1:52              1.0           0.50   

   PULocationID  DOLocationID  
0            60           265  
1           159           168  
2           174           174  
3           136           233  
4           168           247  
Number of duplicate rows in trip data:  662
Number of rows in trip data:  65338
Number of missing values in trip data:  tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          5752
trip_distance              38
PULocationID                0
DOLocationID         

### Data Curation

We now curate hourly taxi demand data with the following steps:

* Attach the exact location information

  We first attach the exact pickup and drop-off location information from the taxi_zone_lookup to trip_dat. This information will be encoded as features for prediction models.




In [109]:
#read the zone lookup data
zone_lookup = pd.read_csv('/Users/laijiang/Documents/Pers/datatest/Data/dat/taxi_zone_lookup.csv')
print(zone_lookup.head())

#only take the first two columns
zone_lookup = zone_lookup.iloc[:, :2]
#show first 10 rows
#print(zone_lookup.head(10))

#merge the trip_dur_dat with the zone lookup data by PULocationID in trip_dur_dat and LocationID in zone_lookup
trip_dur_zone_dat = pd.merge(trip_dur_dat, zone_lookup, left_on='PULocationID', right_on='LocationID', how='left')
#rename the columns
trip_dur_zone_dat.rename(columns={'Borough': 'pickup_borough'}, inplace=True)
#merge the trip_dur_dat with the zone lookup data by DOLocationID in trip_dur_dat and LocationID in zone_lookup
trip_dur_zone_dat = pd.merge(trip_dur_zone_dat, zone_lookup, left_on='DOLocationID', right_on='LocationID', how='left')
#rename the columns
trip_dur_zone_dat.rename(columns={'Borough': 'dropoff_borough'}, inplace=True)

#drop these columns : LocationID_x , LocationID_y, LocationID, Borough, trip_distance_exp
trip_dur_zone_dat = trip_dur_zone_dat.drop(['LocationID_x', 'LocationID_y',  'trip_distance_exp'], axis=1)

#show the first few rows
print("location updated trip data:")
print(trip_dur_zone_dat.head())

#summary statistics of the updated trip data for the columns: pickup_borough, dropoff_borough
print('Summary statistics of pickup_borough: ', trip_dur_zone_dat['pickup_borough'].describe())
print('Summary statistics of dropoff_borough: ', trip_dur_zone_dat['dropoff_borough'].describe())

#missing values of the columns: pickup_borough, dropoff_borough
print('Number of missing values in pickup_borough: ', trip_dur_zone_dat['pickup_borough'].isnull().sum())
print('Number of missing values in dropoff_borough: ', trip_dur_zone_dat['dropoff_borough'].isnull().sum())

#check the DOLocationID of these missing values in dropoff_borough
print('DOLocationID of missing values in dropoff_borough: ', trip_dur_zone_dat[trip_dur_zone_dat['dropoff_borough'].isnull()]['DOLocationID'].unique())

#replace the missing values in dropoff_borough with "outside of NYC"
trip_dur_zone_dat['dropoff_borough'] = trip_dur_zone_dat['dropoff_borough'].fillna('Outside of NYC')

#show the table of counts of dropoff_borough
print('Counts of dropoff_borough: ', trip_dur_zone_dat['dropoff_borough'].value_counts())


   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone
location updated trip data:
  tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0      2023-01-01 0:08       2023-01-01 0:41              1.0          13.75   
1      2023-01-01 0:27       2023-01-01 0:32              1.0           1.03   
2      2023-01-01 0:00       2023-01-01 0:26              1.0           9.30   
3      2023-01-01 1:51       2023-01-01 1:52              1.0           0.50   
4      2023-01-01 1:44       2023-01-01 1:52              4.0           0.90   

   PULocationID  DOLocationID  trip_durations pickup_borough dropoff_borough  
0         

* Hourly Data Binning

  We then attach the hourly time-bin index based on the hour of tpep_pickup_datetime. The idea is to calculate the total number of hours from the start (2023-01-01 00:00) to the end (2023-12-31 23:59), and then assign each row an index corresponding to the hourly time bin it belongs to.


In [110]:
def func_add_time_stamp(df):
    # Convert the tpep_pickup_datetime to datetime format if it's not already
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M')
    # Define the start time
    start_time = pd.Timestamp('2023-01-01 00:00:00')
    # Calculate the number of hours between each pickup time and the start time
    df['pick_up_time_cluster'] = ((df['tpep_pickup_datetime'] - start_time) / np.timedelta64(1, 'h')).astype(int) + 1
    
    #calculate the number of days between each pickup time and the start time
    df['pick_up_day_cluster'] = ((df['tpep_pickup_datetime'] - start_time) / np.timedelta64(1, 'D')).astype(int) + 1
    
    # Return the updated dataframe with the new column
    return df

new_trip_data = func_add_time_stamp(trip_dur_zone_dat)



#sort new_trip_data by the tpep_pickup_datetime
new_trip_data = new_trip_data.sort_values(by='tpep_pickup_datetime')

print(new_trip_data.head(10))

#save for later use
new_trip_data.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/TaxiTrips_cleaned.csv', index=False)



def aggregate_by_time_cluster(df):
    # Convert tpep_pickup_datetime to datetime if it's not already
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M')

    # Extract the hour from the pickup time to add as a feature
    df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
    
    
    # Group by the pick_up_time_cluster
    grouped = df.groupby('pick_up_time_cluster')

    # Create a new DataFrame with aggregated information for each cluster
    result = grouped.agg(
        cluster_ID=('pick_up_time_cluster', 'first'),  # (7) cluster ID
        num_rows=('pick_up_time_cluster', 'size'),  # (1) number of rows in this cluster
        sum_passenger_count=('passenger_count', 'sum'),  # (2) summation of passenger_count
        sum_trip_distance=('trip_distance', 'sum'),  # (3) summation of trip_distance
        sum_trip_durations=('trip_durations', 'sum'),  # (4) summation of trip_durations
        avg_trip_durations=('trip_durations', 'mean'),  # (5) average trip_durations
        pickup_hour=('pickup_hour', 'first'),  # (new feature) the hour of the day for the cluster
        pickup_day = ('pick_up_day_cluster', 'first') # (new feature) the day of the year for the cluster
    ).reset_index(drop=True)

    # Initialize columns for the count of dropoff_borough categories (6)
    borough_columns = ['Bronx', 'Manhattan', 'Queens', 'Brooklyn', 'Outside of NYC', 'Staten Island', 'Unknown', 'EWR']

    # Count the occurrences of each dropoff_borough category within each cluster
    for borough in borough_columns:
        result[borough] = grouped['dropoff_borough'].apply(lambda x: (x == borough).sum()).values
    
    return result

aggregated_data = aggregate_by_time_cluster(new_trip_data)

print("hourly data:")
# Display the first few rows of the aggregated dataframe
print(aggregated_data.head())

#dimension of the aggregated_data
print('Dimension of the aggregated data: ', aggregated_data.shape)

#table of pickup_hour counts nd pickup_day counts
#print('Counts of pickup_hour: ', aggregated_data['pickup_hour'].value_counts())
#print('Counts of pickup_day: ', aggregated_data['pickup_day'].value_counts())



     tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
2     2023-01-01 00:00:00       2023-01-01 0:26              1.0   
0     2023-01-01 00:08:00       2023-01-01 0:41              1.0   
1     2023-01-01 00:27:00       2023-01-01 0:32              1.0   
3663  2023-01-01 00:34:00       2023-01-01 0:55              1.0   
3662  2023-01-01 00:39:00       2023-01-01 0:54              1.0   
3664  2023-01-01 00:41:00       2023-01-01 1:06              1.0   
3661  2023-01-01 00:50:00       2023-01-01 1:14              1.0   
3665  2023-01-01 01:14:00       2023-01-01 1:29              1.0   
4     2023-01-01 01:44:00       2023-01-01 1:52              4.0   
3     2023-01-01 01:51:00       2023-01-01 1:52              1.0   

      trip_distance  PULocationID  DOLocationID  trip_durations  \
2              9.30           136           233            26.0   
0             13.75            60           265            33.0   
1              1.03           159           168   

* Insert missing hours

  Note that some mid-night hours may have no taxi demand. Therefore we will insert rows with 0 taxi demand for these hours. This is necessary for data smoothing later.

In [111]:
#first 

import pandas as pd
import numpy as np
from itertools import product

def insert_missing_rows(df):
    # Create all possible combinations of pickup_day (1 to 365) and pickup_hour (0 to 23)
    all_combinations = pd.DataFrame(list(product(range(1, 366), range(24))), columns=['pickup_day', 'pickup_hour'])

    # Merge with the original data to find missing rows
    merged = pd.merge(all_combinations, df, on=['pickup_day', 'pickup_hour'], how='left')

    # Find missing rows where num_rows is NaN (indicating missing data)
    missing_rows = merged[merged['num_rows'].isna()]

    # Fill missing rows with 0s for all columns except pickup_day and pickup_hour
    for col in df.columns:
        if col not in ['pickup_day', 'pickup_hour']:
            missing_rows.loc[:, col] = 0

    # Combine the original dataframe with the missing rows
    complete_df = pd.concat([df, missing_rows], ignore_index=True)

    # Sort the resulting dataframe by pickup_day and pickup_hour to maintain the correct order
    complete_df = complete_df.sort_values(by=['pickup_day', 'pickup_hour']).reset_index(drop=True)
    
    return complete_df

# Example usage:
filled_data = insert_missing_rows(aggregated_data)

# Display the first few rows of the new dataframe
print(filled_data.head())

#number of rows with cluster_ID = 0
print('Number of rows with cluster_ID = 0: ', filled_data[filled_data['cluster_ID'] == 0].shape[0])
#number of rows with num_rows = 0
#print('Number of rows with num_rows = 0: ', filled_data[filled_data['num_rows'] == 0].shape[0])

#show the first few rows with cluster_ID = 0
#print(filled_data[filled_data['cluster_ID'] == 0].head())

#show the rows from 25 to 30
#print(filled_data[25:30])

#reassign the values of cluster_ID as the row id of the dataframe
filled_data['cluster_ID'] =  filled_data.index

#dimension of filled_data
print('Dimension of filled_data: ', filled_data.shape)

#print the pickup_hour that have 0 num_rows and pickup_day = 1
print('pickup_hour that have 0 num_rows and pickup_day = 2: ', filled_data[(filled_data['num_rows'] == 0) & (filled_data['pickup_day'] == 2)]['pickup_hour'].unique()) 

#print the 
#print('pickup_hour that have 0 num_rows: ', filled_data[filled_data['num_rows'] == 0]['pickup_hour'].unique())

   cluster_ID  num_rows  sum_passenger_count  sum_trip_distance  \
0         1.0       7.0                  7.0             56.230   
1         2.0       4.0                  7.0              9.630   
2         3.0       6.0                  6.0             19.420   
3         4.0       7.0                  8.0             35.690   
4         5.0       8.0                  8.0             36.485   

   sum_trip_durations  avg_trip_durations  pickup_hour  pickup_day  Bronx  \
0               149.0           21.285714            0           1    1.0   
1                40.0           10.000000            1           1    3.0   
2                62.0           10.333333            2           1    1.0   
3               127.0           18.142857            3           1    5.0   
4               194.0           24.250000            4           1    6.0   

   Manhattan  Queens  Brooklyn  Outside of NYC  Staten Island  Unknown  EWR  
0        3.0     1.0       1.0             1.0          

* Data smoothing 

  The zero taxi demand may lead to singularity problems in model fitting and performance evaluation. Thus we need to perform data smoothing on the zero taxi demand hours. Specifically, for each hour where taxi demand  = 0, we take the previous and next hour data, assign the same values, i.e. the round value of average of the three rows, to these three rows for each column of num_rows,sum_passenger_count,  Manhattan,  Queens  Brooklyn  Outside of NYC  Staten Island  Unknown  EWR. For sum_trip_distance and sum_trip_durations we only assign the average vaues of the three rows. For avg_trip_durations, assign the max value of these three rows to all three rows. Note that after this imputation we still have 132 rows with num_rows = 0, which indicates that there exist windows of three hours with taxi demand < 3. For these remaining 132 missing data we replace the zero value with the first non-zero value prior to that hour.



In [112]:
import pandas as pd
import numpy as np

def fill_zero_demand_hours(df):
    # Sort the dataframe by pickup_day and pickup_hour to ensure rows are in the correct order
    df = df.sort_values(by=['pickup_day', 'pickup_hour']).reset_index(drop=True)

    # Iterate through the rows where num_rows == 0
    for i in range(1, len(df) - 1):
        if df.loc[i, 'num_rows'] == 0:
            # Get the previous, current, and next rows
            previous_row = df.loc[i - 1]
            current_row = df.loc[i]
            next_row = df.loc[i + 1]

            # Calculate the rounded average for the columns
            avg_sum_passenger_count = np.round(np.mean([previous_row['sum_passenger_count'], current_row['sum_passenger_count'], next_row['sum_passenger_count']]))
            avg_sum_trip_distance = np.mean([previous_row['sum_trip_distance'], current_row['sum_trip_distance'], next_row['sum_trip_distance']])
            avg_sum_trip_durations = np.mean([previous_row['sum_trip_durations'], current_row['sum_trip_durations'], next_row['sum_trip_durations']])
            avg_num_rows = np.round(np.mean([previous_row['num_rows'], current_row['num_rows'], next_row['num_rows']]))  # Smoothing num_rows

            avg_Manhattan = np.round(np.mean([previous_row['Manhattan'], current_row['Manhattan'], next_row['Manhattan']]))
            avg_Queens = np.round(np.mean([previous_row['Queens'], current_row['Queens'], next_row['Queens']]))
            avg_Brooklyn = np.round(np.mean([previous_row['Brooklyn'], current_row['Brooklyn'], next_row['Brooklyn']]))
            avg_Outside_of_NYC = np.round(np.mean([previous_row['Outside of NYC'], current_row['Outside of NYC'], next_row['Outside of NYC']]))
            avg_Staten_Island = np.round(np.mean([previous_row['Staten Island'], current_row['Staten Island'], next_row['Staten Island']]))
            avg_Unknown = np.round(np.mean([previous_row['Unknown'], current_row['Unknown'], next_row['Unknown']]))
            avg_EWR = np.round(np.mean([previous_row['EWR'], current_row['EWR'], next_row['EWR']]))

            # Calculate the max for avg_trip_durations
            max_avg_trip_durations = np.max([previous_row['avg_trip_durations'], current_row['avg_trip_durations'], next_row['avg_trip_durations']])

            # Assign these values to the previous, current, and next rows
            df.loc[i - 1:i + 1, 'sum_passenger_count'] = avg_sum_passenger_count
            df.loc[i - 1:i + 1, 'sum_trip_distance'] = avg_sum_trip_distance
            df.loc[i - 1:i + 1, 'sum_trip_durations'] = avg_sum_trip_durations
            df.loc[i - 1:i + 1, 'avg_trip_durations'] = max_avg_trip_durations
            df.loc[i - 1:i + 1, 'num_rows'] = avg_num_rows  # Assign smoothed num_rows

            df.loc[i - 1:i + 1, 'Manhattan'] = avg_Manhattan
            df.loc[i - 1:i + 1, 'Queens'] = avg_Queens
            df.loc[i - 1:i + 1, 'Brooklyn'] = avg_Brooklyn
            df.loc[i - 1:i + 1, 'Outside of NYC'] = avg_Outside_of_NYC
            df.loc[i - 1:i + 1, 'Staten Island'] = avg_Staten_Island
            df.loc[i - 1:i + 1, 'Unknown'] = avg_Unknown
            df.loc[i - 1:i + 1, 'EWR'] = avg_EWR

    return df

# Example usage
smoothed_data = fill_zero_demand_hours(filled_data)

# Display the first few rows of the updated dataframe
print(smoothed_data.head())

#print number of rows with num_rows = 0
print('Number of rows with num_rows = 0: ', smoothed_data[smoothed_data['num_rows'] == 0].shape[0])

#the first row with num_rows = 0
#print(smoothed_data[smoothed_data['num_rows'] == 0].head(1))

#print(smoothed_data[95:99])

#for each row with num_rows = 0, we replace the values of the columns with the values of the previous row




def replace_zero_demand_rows(df):
    # Sort the dataframe by pickup_day and pickup_hour to ensure rows are in the correct order
    df = df.sort_values(by=['pickup_day', 'pickup_hour']).reset_index(drop=True)
    
    # Identify rows where num_rows is 0
    zero_demand_indices = df[df['num_rows'] == 0].index
    
    # Replace values in these rows with values from the previous row
    for idx in zero_demand_indices:
        if idx > 0:  # Ensure that we are not trying to access row -1
            df.loc[idx, df.columns != 'pickup_day'] = df.loc[idx - 1, df.columns != 'pickup_day']
            df.loc[idx, df.columns != 'pickup_hour'] = df.loc[idx - 1, df.columns != 'pickup_hour']

    return df

# Example usage:
smoothed_data_final = replace_zero_demand_rows(smoothed_data)

# Display the first few rows of the updated dataframe
#print(smoothed_data_final.head())

#print number of rows with num_rows = 0
#print('Number of rows with num_rows = 0: ', smoothed_data_final[smoothed_data_final['num_rows'] == 0].shape[0])

#print(smoothed_data_final[95:99])

#print("cluster ID adjustment")
#reassign the values of cluster_ID and pickup_hour and pickup_day
smoothed_data_final['cluster_ID'] =  smoothed_data_final.index
smoothed_data_final['pickup_hour'] = smoothed_data_final['cluster_ID'] % 24
smoothed_data_final['pickup_day'] = smoothed_data_final['cluster_ID'] // 24 + 1

#print number of rows with num_rows = 0
#print('Number of rows with num_rows = 0: ', smoothed_data_final[smoothed_data_final['num_rows'] == 0].shape[0])

#print("data dimension: ",smoothed_data_final[95:99])
#print('Dimension of smoothed_data_final: ', smoothed_data.shape)
#print('Counts of pickup_hour: ', smoothed_data['pickup_hour'].value_counts())
#print('Counts of pickup_day: ', smoothed_data['pickup_day'].value_counts())

#save files
smoothed_data_final.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/smoothed_data_final.csv', index=False)



   cluster_ID  num_rows  sum_passenger_count  sum_trip_distance  \
0           0       7.0                  7.0             56.230   
1           1       4.0                  7.0              9.630   
2           2       6.0                  6.0             19.420   
3           3       7.0                  8.0             35.690   
4           4       8.0                  8.0             36.485   

   sum_trip_durations  avg_trip_durations  pickup_hour  pickup_day  Bronx  \
0               149.0           21.285714            0           1    1.0   
1                40.0           10.000000            1           1    3.0   
2                62.0           10.333333            2           1    1.0   
3               127.0           18.142857            3           1    5.0   
4               194.0           24.250000            4           1    6.0   

   Manhattan  Queens  Brooklyn  Outside of NYC  Staten Island  Unknown  EWR  
0        3.0     1.0       1.0             1.0          

* Feature Curation

  We add extra features that may be of interest to the data including:

    * day of the week
    * week of the month
    * holiday index: we mark the public holidays in NYC in the data with a binary variable (1 or 0)
    * weather data from Bronx_Weather_Data2023.csv. We examined the summary statistics of all 4 features and find no apparent outliers.

In [118]:
smoothed_data_final = pd.read_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/smoothed_data_final.csv')


def add_day_of_week(df):
    # Define the base date as 2023-01-01 (which is a Sunday)
    base_date = datetime.date(2023, 1, 1)

    # Create a new column 'day_of_week' by calculating the correct day of the week
    # 1: Monday, 2: Tuesday, ..., 7: Sunday (as per isoweekday())
    df['day_of_week'] = df['pickup_day'].apply(lambda x: (base_date + datetime.timedelta(days=x - 1)).isoweekday())

    return df

# Example usage:
smoothed_data_add = add_day_of_week(smoothed_data_final)

# Display the first few rows of the updated dataframe
#print(smoothed_data_add.head())

#table of counts of day_of_week
#print('Counts of day_of_week: ', smoothed_data_add['day_of_week'].value_counts())


def add_week_of_month(df):
    # Define the base date as 2023-01-01
    base_date = datetime.date(2023, 1, 1)

    # Function to calculate the correct week of the month
    def week_of_month(date):
        return (date.day - 1) // 7 + 1  # This ensures that days 1-7 are week 1, 8-14 are week 2, etc.

    # Create the 'week_of_month' feature by applying the week_of_month function
    df['week_of_month'] = df['pickup_day'].apply(lambda x: week_of_month(base_date + datetime.timedelta(days=x - 1)))

    return df

smoothed_data_add = add_week_of_month(smoothed_data_add)

#print('Counts of week_of_month: ', smoothed_data_add['week_of_month'].value_counts())

#add a binary feature "holiday" that equals to 1 only for holidays
def add_holiday(df):
    # Define the holidays by their pickup_day
    holidays = [1, 2, 16, 44, 51, 99, 134, 149, 169, 170, 185, 247, 282, 311, 315, 327, 359]

    # Create the 'holiday' feature by checking if the pickup_day is in the list of holidays
    df['holiday'] = df['pickup_day'].apply(lambda x: 1 if x in holidays else 0)

    return df

smoothed_data_add = add_holiday(smoothed_data_add)

#print('Counts of holiday: ', smoothed_data_add['holiday'].value_counts())

#print the first few rows
print(smoothed_data_add.head())

#load the weather data for 2023 
weather_data = pd.read_csv('/Users/laijiang/Documents/Pers/datatest/Data/dat/Bronx_Weather_Data2023.csv')

#name the first column as row_index
weather_data = weather_data.rename(columns={'Unnamed: 0': 'row_index'})

#print the first few rows
print(weather_data.head())

#combine weather_data with smoothed_data_add
smoothed_data_add = pd.merge(smoothed_data_add, weather_data, left_on='cluster_ID', right_on='row_index', how='left')

#print(smoothed_data_add.head())

#plot the distribution of temperature_2m
plt.hist(smoothed_data_add['temperature_2m'], bins=30, edgecolor='black')  # Add edgecolor to distinguish bars
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title('Distribution of Temperature')
plt.savefig('temperature.png')
plt.close()


#plot the distribution of precipitation
plt.hist(smoothed_data_add['precipitation'], bins=30)
plt.xlabel('Precipitation')
plt.ylabel('Frequency')
plt.title('Distribution of Precipitation')
plt.savefig('precipitation.png')
plt.close()

#plot the distribution of rain
plt.hist(smoothed_data_add['rain'], bins=30)
plt.xlabel('Rain')
plt.ylabel('Frequency')
plt.title('Distribution of Rain')
plt.savefig('rain.png')
plt.close()

#plot the distribution of snowfall
plt.hist(smoothed_data_add['snowfall'], bins=30)
plt.xlabel('Snowfall')
plt.ylabel('Frequency')
plt.title('Distribution of Snowfall')
plt.savefig('snowfall.png')
plt.close()


#check summary statistics of temperature_2m  precipitation  rain  snowfall  
print('Summary statistics of temperature_2m: ', smoothed_data_add['temperature_2m'].describe())
print('Summary statistics of precipitation: ', smoothed_data_add['precipitation'].describe())
print('Summary statistics of rain: ', smoothed_data_add['rain'].describe())
print('Summary statistics of snowfall: ', smoothed_data_add['snowfall'].describe())

#all looks fine!
#save files
smoothed_data_add.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/smoothed_data_add.csv', index=False)

   cluster_ID  num_rows  sum_passenger_count  sum_trip_distance  \
0           0       7.0                  7.0             56.230   
1           1       4.0                  7.0              9.630   
2           2       6.0                  6.0             19.420   
3           3       7.0                  8.0             35.690   
4           4       8.0                  8.0             36.485   

   sum_trip_durations  avg_trip_durations  pickup_hour  pickup_day  Bronx  \
0               149.0           21.285714            0           1    1.0   
1                40.0           10.000000            1           1    3.0   
2                62.0           10.333333            2           1    1.0   
3               127.0           18.142857            3           1    5.0   
4               194.0           24.250000            4           1    6.0   

   Manhattan  Queens  Brooklyn  Outside of NYC  Staten Island  Unknown  EWR  \
0        3.0     1.0       1.0             1.0         

### Data Modeling

We model the number of hourly taxi demands in the Bronx area and determine the optimal number of taxis required to meet this demand. 

The time-series of the hourly taxi demand is shown in the Figure below. We also show the seasonality of the demand by hour of the day, day of the week, week of the month. The heatmap of hour-of-day and day-of-week shows a clear pattern of higher demand from 8am to 11am on Tuesday to Friday, with a minor outlier at 9pm on Saturday night. This could indicate the demand in Bronx are largely driven by the the work commute. The heatmap of day-of-week and week-of-month shows a clear parttern of higher demand at the first 2 weeks, which could be due to the payroll schedules and its first month effect on consumer behavior (Justine, 2010). 


 We also show the association between demand with holidays, temperature_2m, precipitation,rain and snowfall. The boxplot shows the interesting fact that the holidays have significantly lower demands, which supports the hypothesis that the Bronx taxi demand are largely driven by the work commute. We also observed a signficant negative correlations between temperature and demand, though the relationship can also be seen as nonlinear.  A significant positive correlation between precipitation and demand are also observed. Note that the rain volumn and precipitation are highly corelated (p>0.99). Therefore we dropped the rain volumn as a feature. Finally, the snowfall is not significantly correlated with taxi demand even restricted to snow days (pval=0.33). Therefore we also remove the snowfall features from the data. 

 



In [140]:
#draw the num_rows value against the cluster_ID, colored by the day_of_week
#reduce the size of the dots

# Create the plot with connected lines between the points (omit scatter points)
plt.figure(figsize=(10, 5))
plt.plot(smoothed_data_add['cluster_ID'], smoothed_data_add['num_rows'], 
         color='b')  # 'b' is for blue color line

plt.xlabel('Hour ID')
plt.ylabel('Demands')
plt.title('Demands vs. Hour ID (connected line)')
plt.grid(True)  # Optional: Adds a grid for better visualization

# Save the figure
plt.savefig('Demand_2023_line_plot.png')
plt.close()



# Create the scatter plot with smaller dots (size controlled by 's' parameter)
plt.figure(figsize=(10, 5))
plt.scatter(smoothed_data_add['cluster_ID'], smoothed_data_add['num_rows'], 
            c=smoothed_data_add['day_of_week'], cmap='viridis', s=20)  # 's' controls the size, try reducing to 20

plt.xlabel('Hour ID')
plt.ylabel('Demand')
plt.title('Deman s vs. Hour ID (Colored by Day of Week)')
plt.colorbar(label='Day of Week')

# Save the figure
plt.savefig('Demand_2023_day_of_week.png')
plt.close()


#create a heatmap of the number of rows for each day of the week and hour of the day
# Create a pivot table to aggregate the data
pivot_table = smoothed_data_add.pivot_table(index='day_of_week', columns='pickup_hour', values='num_rows', aggfunc='median')
# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, cmap='viridis')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.title('Median number of demands by Hour and Day')
plt.savefig('heatmap_hour_day.png')
plt.close()

#heat map of the number of rows for each day of the week and week of the month
# Create a pivot table to aggregate the data
pivot_table = smoothed_data_add.pivot_table(index='day_of_week', columns='week_of_month', values='num_rows', aggfunc='median')
# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, cmap='viridis')
plt.xlabel('Week of Month')
plt.ylabel('Day of Week')
plt.title('Median number of demands by Week and Day')
plt.savefig('heatmap_week_day.png')
plt.close()

#plot the boxplot of the number of rows for binary variable holiday
plt.figure(figsize=(10, 5))
sns.boxplot(x='holiday', y='num_rows', data=smoothed_data_add)
plt.xlabel('Holiday')
plt.ylabel('Number of Demands')
plt.title('Number of Demands on Holidays vs. Non-Holidays')
plt.savefig('boxplot_holiday.png')
plt.close()
#test for the difference of the number of rows between holidays and non-holidays
from scipy.stats import ttest_ind
#perform the t-test
holiday = smoothed_data_add[smoothed_data_add['holiday'] == 1]['num_rows']
non_holiday = smoothed_data_add[smoothed_data_add['holiday'] == 0]['num_rows']
t_stat, p_value = ttest_ind(holiday, non_holiday)
print('P-value for t-test of demand vs holiday (yes or no):', p_value)

#draw the scatter plot of the number of rows against the temperature_2m
plt.figure(figsize=(10, 5))
plt.scatter(smoothed_data_add['temperature_2m'], smoothed_data_add['num_rows'], s=20)
plt.xlabel('Temperature')
plt.ylabel('Number of Demands')
plt.title('Number of Demands vs. Temperature')
plt.savefig('demand_temperature.png')
plt.close()

#test for the correlation between the number of rows and the temperature_2m
from scipy.stats import pearsonr
#perform the t-test
corr, p_value = pearsonr(smoothed_data_add['temperature_2m'], smoothed_data_add['num_rows'])
print('Correlation between temperature and demand:', corr)
print('P-value for correlation between temperature and demand:', p_value)

#plot the scatter plot of the number of rows against the precipitation
plt.figure(figsize=(10, 5))
plt.scatter(smoothed_data_add['precipitation'], smoothed_data_add['num_rows'], s=20)
plt.xlabel('Precipitation')
plt.ylabel('Number of Demands')
plt.title('Number of Demands vs. Precipitation')
plt.savefig('demand_precipitation.png')
plt.close()

#test for the correlation between the number of rows and the precipitation
#perform the t-test
corr, p_value = pearsonr(smoothed_data_add['precipitation'], smoothed_data_add['num_rows'])
print('Correlation between precipitation and demand:', corr)
print('P-value for correlation between precipitation and demand:', p_value)

#plot the scatter plot of the number of rows against the rain
plt.figure(figsize=(10, 5))
plt.scatter(smoothed_data_add['rain'], smoothed_data_add['num_rows'], s=20)
plt.xlabel('Rain')
plt.ylabel('Number of Demands')
plt.title('Number of Demands vs. Rain')
plt.savefig('demand_rain.png')
plt.close()

#correlation between the train and precipitation
corr, p_value = pearsonr(smoothed_data_add['rain'], smoothed_data_add['precipitation'])
print('Correlation between rain and precipitation:', corr)
print('P-value for correlation between rain and precipitation:', p_value)

#plot the scatter plot of the number of rows against the snowfall
plt.figure(figsize=(10, 5))
plt.scatter(smoothed_data_add['snowfall'], smoothed_data_add['num_rows'], s=20)
plt.xlabel('Snowfall')
plt.ylabel('Number of Demands')
plt.title('Number of Demands vs. Snowfall')
plt.savefig('demand_snowfall.png')
plt.close()

#test for the correlation between the number of rows and the snowfall
#perform the t-test
corr, p_value = pearsonr(smoothed_data_add['snowfall'], smoothed_data_add['num_rows'])
print('Correlation between snowfall and demand:', corr)
print('P-value for correlation between snowfall and demand:', p_value)

#test for the correlation between the number of rows and the snowfall only for the days with snowfall > 0
#perform the t-test
corr, p_value = pearsonr(smoothed_data_add[smoothed_data_add['snowfall'] > 0]['snowfall'], smoothed_data_add[smoothed_data_add['snowfall'] > 0]['num_rows'])
print('Correlation between snowfall and demand on snowy days:', corr)
print('P-value for correlation between snowfall and demand on snowy days:', p_value)

#remove the snowfall and rain columns
hourly_data_final = smoothed_data_add.drop(['snowfall', 'rain'], axis=1)

#save data
hourly_data_final.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/hourly_data_final.csv', index=False)


P-value for t-test of demand vs holiday (yes or no): 4.2340728242108675e-07
Correlation between temperature and demand: -0.12419473373647003
P-value for correlation between temperature and demand: 1.8677415379381947e-31
Correlation between precipitation and demand: 0.019080769587125818
P-value for correlation between precipitation and demand: 0.07413652390932238
Correlation between rain and precipitation: 0.9975879846479938
P-value for correlation between rain and precipitation: 0.0
Correlation between snowfall and demand: -0.0017976350734163454
P-value for correlation between snowfall and demand: 0.8664058412900063
Correlation between snowfall and demand on snowy days: 0.10634895887400495
P-value for correlation between snowfall and demand on snowy days: 0.3356310767014079


#### Baseline Model

The most intuitive modeling of the time-series demands is to use the previous houly demands to predict the future demand. Considering the strong cyclical nature of houly taxi demand, we adop the weighted moving average model as the baseline model for this task. Note that we prefer weighted moving average over simple moving average because of the assumption that taxi demand at a specific time t is more likely to be close to the demand at t-1 than that from a more distant timestamp. 

We choose Mean Absolute Percentage Error (MAPE) as the performance evaluation metric for this analysis. Among all performance evaluation metrics, including mean square errors (MSE), mean absolute errors (MAE) and root mean squared errors (RMSE), we prefer MAPE for the following reasons:  

  * (i) it is more sensitive to larger percentage errors, therefore ideal to evaluate model performance during low-demand periods in order to reduce operational cost of idle vehicles; 
  * (ii) the Bronx data here has no apparent outlier demand values, i.e. the high demand period data are reliable. Therefore the sensitivity of MAPE to larger percentage error is a desirable property to favor models with accuate predictions of the high demand and thus high revenue periods;  
  * (iii) it is widely accepted in the literature of traffic predictions for the reasons listed above.

We decided the best baseline model is WMA with window size equals to 2, after manually tunning of the chocie of window size to achieve best MAPE values on the complete data. The final WMA model achieved the MAPE with 0.18 as below. We will use this result as baseline to improve our prediction models.

In [176]:

# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) 

# Function to compute the Weighted Moving Average (WMA)
def func_wma(data, window_size):
    weights = np.arange(1, window_size + 1)  # Weights are 1, 2, ..., window_size
    wma = data.rolling(window_size).apply(lambda values: np.dot(values, weights) / weights.sum(), raw=True)
    wma = wma.round(0)  # Round the WMA predictions
    wma = wma.dropna()  # Drop NaN values 
    #drop the last value of wma
    wma = wma[:-1]
    data = data[window_size:]  # Drop the first few rows to match the length 
    mape = mean_absolute_percentage_error(data, wma)
    return mape


window_size = 2  # You can adjust this as needed

# Calculate WMA for num_rows
wma2 = func_wma(hourly_data_final['num_rows'], window_size)
#print the MAPE
print('Baseline model: WMA2, WMA with window size 2')
print('MAPE of baseline model WMA2:', wma2.round(4))







Baseline model: WMA2, WMA with window size 2
MAPE of baseline model WMA2: 0.1813


#### Machine Learning Models


There has been extensive publications that adopt machine learning algorithms to pursue taxi demand predictions. The advantage of using machine learning algorithm over moving average methods includes:

  * It has the potential to incoporate features that reflect the cyclical and seasonal patterns. 
    * Fourier transformation is a widely accepted method to model cyclical patterns and nested seasonalities, of which the frequency can be incorporated into machine learning algorithm as predictive features in comman practise. 
    * Seasonal Decomposition of Time Series (STL) can also be used to capture long-term trends and seasonal cycles.
    * Holt-Winters' Seasonal method is also suitable for our analysis since it captures the stable seasonlaities with fixed periods (e.g. hour, day, week). 
    * Other transformation methods such as Wavelet transform and box-cox transformation may be of interest but unlikely to have significant boost to our analysis due to their nature. Therefore we will not consider these transformations in this analysis.

  * It allow more flexible weighting strategy for the utilization of the lagged features. In fact, WMA assigns the historical taxi demand at t-1, t-2..t-N  a weight vector with fixed values (e.g. N,N-1,..1) for predicting taxi demand at time-point t. In contrast, machine learning algorithms can incorporate historical taxi demand as input features and obtain data-driven estimations of the weight vector to understand time dependencies.

  * It allows the incorporating of additional features such as weather, spatial factors (drop-off locations)...etc.

  * There is a wide range of machine learning algorithms, with diverse underlying mechanisms, to compare and choose.



##### Improved models

Before we make selections of machine learning algorithm, we conduct extra feature curation to address the oppertunities above:


data split: 

  we split the time-series data into training and test set. The training data contains data of January to September, the test data contains data from October to December. Note that we intentionally put the September 2023 into the training set to learn the patterns in September, such that the prediction of September 2024 would not miss important signals.


Fourier transformation:

  we fist examine the fourier features on training data. The figure shows there were 5 peaks on amplitude vs frequency plot. The highest peak is from DC component. The rest peaks are according to frequency: 1/24, 1/(24*7), 1/12, 1/28. Thus we incorporate the 11 fourier features from the top 6 signals, while reomving the DC component. The amplitude and frequency are extracted for each month (Jan, Feb....etc) seperately.








In [209]:

#save the training and testing data
#train_data.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/train_data.csv', index=False)
#test_data.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/test_data.csv', index=False)

#Feature curation 1: Fourier transformation
from scipy.fft import fft, fftfreq


# Function to apply Fourier transformation and draw scatterplot of Frequency vs. Amplitude
def fourier_transform_scatterplot(time_series):
    N = len(time_series)  # Length of the time series
    time_series = np.array(time_series)  # Ensure the data is in the correct format
    fft_values = fft(time_series)  # Compute the Fourier transform
    frequencies = fftfreq(N, 1)  # Get corresponding frequencies
    
    # Compute Amplitudes
    amplitude = np.abs(fft_values)  # Magnitude of the Fourier coefficients
    
    # Limit to Positive Frequencies
    positive_freq_idx = np.where(frequencies >= 0)  # Consider only positive frequencies
    frequencies = frequencies[positive_freq_idx]
    amplitude = amplitude[positive_freq_idx]
    
    # Plot the scatterplot of Frequency vs. Amplitude
    plt.figure(figsize=(10, 6))
    plt.scatter(frequencies, amplitude, color='blue')
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.title('Fourier Transform')
    plt.grid(True)
    plt.savefig("Fourier.png")
    plt.close()
    
    return frequencies, amplitude

# Apply the Fourier transformation and get frequencies and amplitudes
frequencies, amplitude = fourier_transform_scatterplot(train_data['num_rows'])

# Print the top 10 amplitudes and their corresponding frequencies
# Sort the amplitudes in descending order
sorted_indices = np.argsort(amplitude)[::-1]
# Get the top 10 frequencies and amplitudes
top_10_freq = frequencies[sorted_indices][:10]
top_10_amplitude = amplitude[sorted_indices][:10]
# Print the top 10 frequencies and amplitudes
print('Top 10 Frequencies:', top_10_freq)
print('Top 10 Amplitudes:', top_10_amplitude)




Top 10 Frequencies: [0.         0.04174821 0.04158513 0.08333333 0.03571429 0.00603392
 0.00587084 0.04761905 0.04191129 0.04142205]
Top 10 Amplitudes: [41409.         11368.53957498 11048.33794188  6206.10059245
  5800.10714177  4445.76102562  3826.36509206  3825.88049372
  3533.00381812  3474.36674358]


In [218]:
#add a feature: month to the hourly_data_final.
def add_month(df):
    # Define the base date as 2023-01-01
    base_date = datetime.date(2023, 1, 1)

    # Create a new column 'month' by calculating the correct month
    df['month'] = df['pickup_day'].apply(lambda x: (base_date + datetime.timedelta(days=x - 1)).month)

    return df

# Example usage:
hourly_data_month = add_month(hourly_data_final)



# Initialize an empty DataFrame to store the final results with Fourier features
fourier_features_final = pd.DataFrame()

# Function to extract top N Fourier features for a time series
def extract_top_fourier_features(time_series, top_n=5):

    fft_values = np.fft.fft(time_series)
    frequencies = np.fft.fftfreq(len(time_series), 1)
    
    # Compute amplitudes
    amplitude = np.abs(fft_values)
    
    # Sort the amplitudes in descending order to get the top N values
    sorted_indices = np.argsort(amplitude)[::-1]
    
    # Get the top N frequencies and amplitudes
    top_frequencies = frequencies[sorted_indices][:top_n]
    top_amplitudes = amplitude[sorted_indices][:top_n]

    # Create a dictionary to hold the features
    features = {}
    for i in range(top_n):
        features[f'f_{i+1}'] = top_frequencies[i]
        features[f'a_{i+1}'] = top_amplitudes[i]
    
    return features

# Iterate over each month (1 to 12)
for month in range(1, 13):
    # Extract the data for the current month
    monthly_data = hourly_data_month[hourly_data_month['month'] == month].copy()

    # Extract the 'num_rows' time series for Fourier Transformation
    num_rows_series = monthly_data['num_rows'].values
    
    # Apply Fourier Transformation and extract top 5 features for this month
    fourier_features = extract_top_fourier_features(num_rows_series, top_n=5)
    
    # Convert the dictionary of features to a DataFrame and replicate it for all rows in this month
    features_df = pd.DataFrame([fourier_features] * len(monthly_data))
    
    # Concatenate the features with the original data for the current month
    monthly_with_features = pd.concat([monthly_data.reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)
    
    # Append the results to the final DataFrame for all months
    fourier_features_final = pd.concat([fourier_features_final, monthly_with_features], ignore_index=True)


# Display the first few rows of the final DataFrame with Fourier features
#print(fourier_features_final.head())

#drop the f1 feature
fourier_features_final = fourier_features_final.drop(['f_1'], axis=1)

#print the count of unique features of a_1
#print('Counts of a_1: ', fourier_features_final['a_1'].value_counts())


Counts of unique features of a_1:  a_1
3911.0    744
5424.0    744
5363.0    744
4864.0    744
5857.0    744
6033.0    744
6553.0    744
4857.0    720
4827.0    720
5334.0    720
5840.0    720
3946.0    672
Name: count, dtype: int64


STL decomposition:

  We extract features from STL decomposition to capture the trend and seasonality. Three seasonal period is considered: semi-daily (12), daily (24), weekly (24*7). We applied STL to each month seperately and combined the seasonal, trend and residuals features together. In total we created 9 STL features (i.e. seasonal, trend and residuals for each of the three periods).

  

In [251]:
from statsmodels.tsa.seasonal import STL
import pandas as pd

# Function to perform STL decomposition and extract features
def extract_stl_features(time_series, seasonal_period=25):
    stl = STL(time_series, seasonal=seasonal_period, period=seasonal_period)
    result = stl.fit()
    
    # Create a DataFrame to store the extracted features
    stl_features = pd.DataFrame({
        'trend': result.trend,
        'seasonal': result.seasonal,
        'residual': result.resid
    })
    
    return stl_features

# Function to perform STL decomposition and extract features for each month
def extract_stl_features_by_month(data, target_column, seasonal_period=25):

    stl_features_final = pd.DataFrame()  # To store the final result
    
    # Loop through each unique month
    for month in range(1, 13):
        # Extract the subset of data for the current month
        monthly_data = data[data['month'] == month].copy()
        
        # Perform STL decomposition on the target column (e.g., 'num_rows')
        time_series = monthly_data[target_column].values
        
        if len(time_series) >= seasonal_period:
            # Perform STL decomposition
            stl_features = extract_stl_features(time_series, seasonal_period)
            
            # Append the results for this month to the final DataFrame
            stl_features_final = pd.concat([stl_features_final, stl_features], ignore_index=True)
    
    return stl_features_final

#stl features for the daily, semi-daily and semi-weekly seasonal periods
stl_features_daily = extract_stl_features_by_month(fourier_features_final, target_column='num_rows', seasonal_period=25)
stl_features_semi_daily = extract_stl_features_by_month(fourier_features_final, target_column='num_rows', seasonal_period=13)
stl_features_semi_weekly = extract_stl_features_by_month(fourier_features_final, target_column='num_rows', seasonal_period=169)

#combine
stl_features_final = pd.concat([stl_features_daily, stl_features_semi_daily, stl_features_semi_weekly], axis=1)
#print dimension
print('Dimension of stl_features_final: ', stl_features_final.shape)

#the column names as "STL_trend_daily", "STL_seasonal_daily", "STL_residual_daily", "STL_trend_semi_daily", "STL_seasonal_semi_daily", "STL_residual_semi_daily", "STL_trend_semi_weekly", "STL_seasonal_semi_weekly", "STL_residual_semi_weekly"
stl_features_final.columns = ['STL_trend_daily', 'STL_seasonal_daily', 'STL_residual_daily', 'STL_trend_semi_daily', 'STL_seasonal_semi_daily', 'STL_residual_semi_daily', 'STL_trend_semi_weekly', 'STL_seasonal_semi_weekly', 'STL_residual_semi_weekly']

#now add these features to the fourier_features_final
stl_fourier_features = pd.concat([fourier_features_final, stl_features_final], axis=1)
#print column names of stl_fourier_features
print(stl_fourier_features.columns)


Dimension of stl_features_final:  (8760, 9)
Index(['cluster_ID', 'num_rows', 'sum_passenger_count', 'sum_trip_distance',
       'sum_trip_durations', 'avg_trip_durations', 'pickup_hour', 'pickup_day',
       'Bronx', 'Manhattan', 'Queens', 'Brooklyn', 'Outside of NYC',
       'Staten Island', 'Unknown', 'EWR', 'day_of_week', 'week_of_month',
       'holiday', 'row_index', 'date', 'temperature_2m', 'precipitation',
       'month', 'a_1', 'f_2', 'a_2', 'f_3', 'a_3', 'f_4', 'a_4', 'f_5', 'a_5',
       'STL_trend_daily', 'STL_seasonal_daily', 'STL_residual_daily',
       'STL_trend_semi_daily', 'STL_seasonal_semi_daily',
       'STL_residual_semi_daily', 'STL_trend_semi_weekly',
       'STL_seasonal_semi_weekly', 'STL_residual_semi_weekly'],
      dtype='object')


Holt-Winters’ (HW) seasonal method

  Also know as Triple Exponential Smoothing, the Holt-Winters' seasonal method is popular for capturing the level, trend and seasonal components. The major benifits of HW method is it is able to capture and adjsuts for trend over-time, which Fourier transformation could not. It is also been proven that it works excellent for tiem-series with regular seasonalities (e.g. day and week). Thus we also extract the HW trend, level, and seasonal features for each month by re-using the code from Gregory Trubetskoy (https://grisha.org/blog/2016/02/17/triple-exponential-smoothing-forecasting-part-iii/). Note that HW method has three parameters alpha, beta and gamma, which controlls different levels of smoothing on level, trend and seasonality. We consider 27 combinations of possible parameter values, i.e. alpha, beta, gamma =  (0.1, 0.5, 0.9) and 2 seasonlities: daily and weekly. Therefore we will have 54 = 2*27 HW features (ie. predictions) extracted from training and test set seperately.


  

In [345]:


def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen

def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals


def triple_exponential_smoothing(series, slen, alpha, beta, gamma, n_preds):
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result

In [464]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Example function to perform GridSearch for HW parameters on training data using TimeSeriesSplit
def tune_hw_params_custom_cv(train_data, seasonal_length, target_column='num_rows', n_splits=5):
    # Define the parameter grid for alpha, beta, gamma
    param_grid = {
        'alpha': [ 0.1, 0.3, 0.9],
        'beta': [  0.1, 0.3, 0.9],
        'gamma': [ 0.1, 0.3, 0.9],
    }
    
    best_params = None
    best_score = float('inf')
    
    # Time series split for cross-validation
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Loop through each combination of alpha, beta, and gamma
    for alpha in param_grid['alpha']:
        for beta in param_grid['beta']:
            for gamma in param_grid['gamma']:
                mse_scores = []
                
                # Perform cross-validation using TimeSeriesSplit
                for train_index, val_index in tscv.split(train_data):
                    train_fold = train_data.iloc[train_index]
                    val_fold = train_data.iloc[val_index]

                    try:
                        # Fit model on the training fold
                        fitted_values = triple_exponential_smoothing(
                            train_fold[target_column].values, seasonal_length, alpha, beta, gamma, n_preds=0
                        )
                        
                        # Forecast only for the length of the validation fold
                        val_preds = triple_exponential_smoothing(
                            train_fold[target_column].values, seasonal_length, alpha, beta, gamma, n_preds=len(val_fold)
                        )[-len(val_fold):]

                        # Ensure the prediction length matches the validation set length
                        assert len(val_preds) == len(val_fold), f"Forecast length {len(val_preds)} does not match validation set length {len(val_fold)}"

                        # Calculate MSE on the validation fold
                        mse = mean_squared_error(val_fold[target_column].values, val_preds)
                        #mae = mean_absolute_error(val_fold[target_column].values, val_preds)
                        #mape = mean_absolute_percentage_error(val_fold[target_column].values, val_preds)
                        mse_scores.append(mse)

                    except Exception as e:
                        print(f"Model failed with alpha={alpha}, beta={beta}, gamma={gamma}: {str(e)}")
                        continue

                # Calculate the average MSE across folds
                avg_mse = np.mean(mse_scores)

                # Keep track of the best parameters
                if avg_mse < best_score:
                    best_score = avg_mse
                    best_params = (alpha, beta, gamma)
    
    return best_params


def apply_hw_with_best_params_custom(train_data, test_data, best_params, seasonal_lengths, target_column='num_rows'):
    alpha, beta, gamma = best_params
    
    for seasonal_length in seasonal_lengths:
        # Apply triple exponential smoothing to training data
        train_data[f'hw_{seasonal_length}'] = triple_exponential_smoothing(
            train_data[target_column].values, seasonal_length, alpha, beta, gamma, n_preds=0
        )
        
        # Forecast on test data
        forecast = triple_exponential_smoothing(
            train_data[target_column].values, seasonal_length, alpha, beta, gamma, n_preds=len(test_data)
        )[-len(test_data):]  # Take only the forecasted part
        
        # Store forecast in test data
        test_data[f'hw_{seasonal_length}'] = forecast
    
    return train_data, test_data


# Assuming 'hourly_data_final' contains your time series data and 'num_rows' is the target column
train_data = stl_fourier_features[stl_fourier_features['month'] <= 9]  # Jan to Sep for training
test_data = stl_fourier_features[stl_fourier_features['month'] > 9]  # Oct to Dec for testing

# Reset the index
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Tune the HW parameters using time series cross-validation
best_params_168 = tune_hw_params_custom_cv(train_data, seasonal_length=168, target_column='num_rows', n_splits=5)
best_params_24  = tune_hw_params_custom_cv(train_data, seasonal_length=24, target_column='num_rows', n_splits=5)

# Print best parameters
print('Best parameters for seasonal_length=168:', best_params_168)
print('Best parameters for seasonal_length=24:', best_params_24)

# Apply Holt-Winters with tuned parameters to train and test data
train_data, test_data = apply_hw_with_best_params_custom(train_data, test_data, best_params_168, [168])
train_data, test_data = apply_hw_with_best_params_custom(train_data, test_data, best_params_24, [24])

# Check the result
print(f"Number of NaN values in training data: {train_data.isna().sum().sum()}")
print(f"Number of NaN values in test data: {test_data.isna().sum().sum()}")


Best parameters for seasonal_length=168: (0.1, 0.1, 0.1)
Best parameters for seasonal_length=24: (0.1, 0.1, 0.1)
Number of NaN values in training data: 0
Number of NaN values in test data: 0


Final predictor list

  * Lagged demand: 
      We also include the lagged taxi demand for each time-point as predictors. i.e. for each hour t, the number of taxi demand at t-1, t-2, t-3 are included as predictors.

  * Lagged passenger counts. from the previous first, second and third hours.

  * Lagged trip durations. for the last three hours.

  * Lagged Bronx drop-offs.

  * lagged Manhattan drop-offs.

  * day_of_week.

  * week_of_month.

  * holiday 

  * temperature_2m
  
  * precipitation
  
  * Fourier features: amplitude a_1, frequencies, f_2....

  * STL features: stl_...

  * Holt Winters features: hw_...




In [465]:
import pandas as pd

#check missing or NA vlalues in train_data and test_data
print('Number of rows with NA values in test_data: ', test_data.isnull().sum().sum())
print('Number of rows with NA values in train_data: ', train_data.isnull().sum().sum())


#create a new feature demand_1 that is the demand of the previous hour, demand_2 that is the demand of the hour before the previous hour, ... until demand_5.
def add_demand_lags(df, num_lags=5):
    for i in range(1, num_lags + 1):
        df[f'lagged_demand_{i}'] = df['num_rows'].shift(i)
    return df

#add lagged sum_passenger_count similarly
def add_sum_passenger_count_lags(df, num_lags=5):
    for i in range(1, num_lags + 1):
        df[f'lagged_sum_passenger_count_{i}'] = df['sum_passenger_count'].shift(i)
    return df

#add lagged sum_trip_durations similarly
def add_sum_trip_durations_lags(df, num_lags=5):
    for i in range(1, num_lags + 1):
        df[f'lagged_sum_trip_durations_{i}'] = df['sum_trip_durations'].shift(i)
    return df

#add lagged Bronx drop-offs similarly
def add_Bronx_lags(df, num_lags=5):
    for i in range(1, num_lags + 1):
        df[f'lagged_Bronx_{i}'] = df['Bronx'].shift(i)
    return df

#add lagged Manhattan drop-offs similarly
def add_Manhattan_lags(df, num_lags=5):
    for i in range(1, num_lags + 1):
        df[f'lagged_Manhattan_{i}'] = df['Manhattan'].shift(i)
    return df



train_data_lag = add_demand_lags(train_data)
test_data_lag = add_demand_lags(test_data)

train_data_lag = add_sum_passenger_count_lags(train_data_lag)
test_data_lag = add_sum_passenger_count_lags(test_data_lag)

train_data_lag = add_sum_trip_durations_lags(train_data_lag)
test_data_lag = add_sum_trip_durations_lags(test_data_lag)


train_data_lag = add_Bronx_lags(train_data_lag)
test_data_lag = add_Bronx_lags(test_data_lag)


train_data_lag = add_Manhattan_lags(train_data_lag)
test_data_lag = add_Manhattan_lags(test_data_lag)



#save the train_data_lag and test_data_lag
train_data_lag.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/train_data_lag.csv', index=False)
test_data_lag.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/test_data_lag.csv', index=False)


#print('Number of rows with NA values in test_data: ', test_data.isnull().sum().sum())  
#print('Number of rows with NA values in train_data: ', train_data.isnull().sum().sum())  
#drop rows with NA values in train_data_lag and test_data_lag
train_data_lag = train_data_lag.dropna()
test_data_lag = test_data_lag.dropna()
#print dimensio
train_data_lag.shape
#print('Dimension of test_data_lag: ', test_data_lag.shape)
#print(train_data_lag[['num_rows', 'demand_1']].head(10))

#now select columns:
#* day_of_week  week_of_month holiday temperature_2m precipitation  
# a_1, a_2,..any feture starts with a_ 
# any feature starts with f_ 
# columns start with stl_...
# #columns start with hw_...
# columns start with lagged_

# Define the columns to keep
selected_columns = ['num_rows','day_of_week', 'week_of_month', 'holiday', 'temperature_2m', 'precipitation']
selected_columns += [col for col in train_data_lag.columns if col.startswith('a_')]
selected_columns += [col for col in train_data_lag.columns if col.startswith('f_')]
selected_columns += [col for col in train_data_lag.columns if col.startswith('STL_')]
selected_columns += [col for col in train_data_lag.columns if col.startswith('hw_')]
selected_columns += [col for col in train_data_lag.columns if col.startswith('lagged_')]
# Select the columns in the training and testing data
train_data_selected = train_data_lag[selected_columns]
test_data_selected = test_data_lag[selected_columns]

#dimension
print('Dimension of train_data_selected: ', train_data_selected.shape)
print('Dimension of test_data_selected: ', test_data_selected.shape)

#check NA or missing values in train_data_selected and test_data_selected
#print('Number of rows with NA values in train_data_selected: ', train_data_selected.isnull().sum().sum())
#print('Number of rows with NA values in test_data_selected: ', test_data_selected.isnull().sum().sum())

#save data
train_data_selected.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/train_data_selected.csv', index=False)
test_data_selected.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/test_data_selected.csv', index=False)

#print the column names of train_data_selected
print(train_data_selected.columns)


Number of rows with NA values in test_data:  0
Number of rows with NA values in train_data:  0
Dimension of train_data_selected:  (6547, 51)
Dimension of test_data_selected:  (2203, 51)
Index(['num_rows', 'day_of_week', 'week_of_month', 'holiday', 'temperature_2m',
       'precipitation', 'a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'f_2', 'f_3', 'f_4',
       'f_5', 'STL_trend_daily', 'STL_seasonal_daily', 'STL_residual_daily',
       'STL_trend_semi_daily', 'STL_seasonal_semi_daily',
       'STL_residual_semi_daily', 'STL_trend_semi_weekly',
       'STL_seasonal_semi_weekly', 'STL_residual_semi_weekly', 'hw_168',
       'hw_24', 'lagged_demand_1', 'lagged_demand_2', 'lagged_demand_3',
       'lagged_demand_4', 'lagged_demand_5', 'lagged_sum_passenger_count_1',
       'lagged_sum_passenger_count_2', 'lagged_sum_passenger_count_3',
       'lagged_sum_passenger_count_4', 'lagged_sum_passenger_count_5',
       'lagged_sum_trip_durations_1', 'lagged_sum_trip_durations_2',
       'lagged_sum_trip_du

Improved Model I

We first start with a simple random forest method and find the RandomForest with STL features can achieve MAPE = 0.15. Interestingly, the random forest seems to converge to a RF model with only HW features when all features avaialbe, or features with only STL features if HW features are removed. This implies that (i) HW Features are highly predictive; and (ii) W and STL feature groups may have multicollinearity, (iii) the RF could be overfitting on training data due to the number of features. In order to address these issues, we will explore model improvement.



In [475]:
#build a random forest model on train_data_selected for predicting the num_rows
from sklearn.ensemble import RandomForestRegressor

# Load the training data
train_data_selected = pd.read_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/train_data_selected.csv')

# Define the target column
target_column = 'num_rows'

# Split the data into features and target
X_train = train_data_selected.drop(target_column, axis=1)
y_train = train_data_selected[target_column]


#remove these columns from HW transformation with extremely high values, they are due to mistaches of 
#the manually selection of alpha, beta, gamma
#rm_columns = X_train.columns[(X_train > 10e6).any()]
#X_train = X_train.drop(rm_columns, axis=1)



rf_model = RandomForestRegressor(n_estimators=100, random_state=50,n_jobs=-1)


# Fit the model on the training data
rf_model.fit(X_train, y_train)

# Load the testing data
test_data_selected = pd.read_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/test_data_selected.csv')

# Split the data into features and target
X_test = test_data_selected.drop(target_column, axis=1)
y_test = test_data_selected[target_column]
#X_test = X_test.drop(rm_columns, axis=1)

# Make predictions on the test data
predictions = rf_model.predict(X_test)
predictions_tr = rf_model.predict(X_train)

# Calculate the MAPE on the test data
mape_IM1 = mean_absolute_percentage_error(y_test, predictions)
print('MAPE of the Random Forest model:', round(mape_IM1, 4))

mape_train = mean_absolute_percentage_error(y_train, predictions_tr)

print('MAPE of the Random Forest on training:', round(mape_train, 4))

#plot the importance
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]



indices_drop = indices[:10]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importance of Random Forest Model without Holt-Winters")
plt.barh(range(len(indices_drop)), importances[indices_drop], align="center")
plt.yticks(range(len(indices_drop)), X_train.columns[indices_drop])
plt.xlabel("Relative Importance")
plt.gca().invert_yaxis()  # Invert y-axis to have the highest importance on top
plt.tight_layout()

# Save the plot as an image
plt.savefig("imp_model1.png")
plt.close()


MAPE of the Random Forest model: 0.1936
MAPE of the Random Forest on training: 0.0362


In [395]:
# Feature importance from the random forest model
importances = rf_model.feature_importances_

# Sort the feature importances in descending order
indices = np.argsort(importances)[::-1]

# show the top 20 features
indices = indices[:20]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importance of Random Forest Model")
plt.barh(range(len(indices)), importances[indices], align="center")
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.xlabel("Relative Importance")
plt.gca().invert_yaxis()  # Invert y-axis to have the highest importance on top
plt.tight_layout()

# Save the plot as an image
plt.savefig("imp_model1.png")
plt.close()


#collect the subset of  of the y_test and column  hw_168_19
#then save to a csv file
y_test_hw = y_test
hw_168_19 = X_test['hw_168']
y_test_hw = pd.concat([y_test_hw, hw_168_19], axis=1)
y_test_hw.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/y_test_hw.csv', index=False)


In [476]:
#run a new random forest by dropping all hw_.. columns
# Define the columns to drop
columns_to_drop = [col for col in X_train.columns if col.startswith('hw_')]
# Drop the columns from the training and testing data
X_train_drop_hw = X_train.drop(columns_to_drop, axis=1)
X_test_drop_hw = X_test.drop(columns_to_drop, axis=1)

#print the column names of X_train_drop_hw
print(X_train_drop_hw.columns)

# Initialize a new Random Forest model
rf_model_drop_hw = RandomForestRegressor(n_estimators=100, random_state=50,n_jobs=-1)

# Fit the model on the training data
rf_model_drop_hw.fit(X_train_drop_hw, y_train)

# Make predictions on the test data
predictions_drop_hw = rf_model_drop_hw.predict(X_test_drop_hw)

# Calculate the MAPE on the test data
mape_IM2 = mean_absolute_percentage_error(y_test, predictions_drop_hw)
print('MAPE of the Random Forest model without Holt-Winters:', round(mape_IM2, 4))

# Feature importance from the random forest model without Holt-Winters
importances_drop_hw = rf_model_drop_hw.feature_importances_

# Sort the feature importances in descending order
indices_drop_hw = np.argsort(importances_drop_hw)[::-1]

# show the top 20 features
indices_drop_hw = indices_drop_hw[:20]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importance of Random Forest Model without Holt-Winters")
plt.barh(range(len(indices_drop_hw)), importances_drop_hw[indices_drop_hw], align="center")
plt.yticks(range(len(indices_drop_hw)), X_train_drop_hw.columns[indices_drop_hw])
plt.xlabel("Relative Importance")
plt.gca().invert_yaxis()  # Invert y-axis to have the highest importance on top
plt.tight_layout()

# Save the plot as an image
plt.savefig("imp_model2.png")
plt.close()


Index(['day_of_week', 'week_of_month', 'holiday', 'temperature_2m',
       'precipitation', 'a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'f_2', 'f_3', 'f_4',
       'f_5', 'STL_trend_daily', 'STL_seasonal_daily', 'STL_residual_daily',
       'STL_trend_semi_daily', 'STL_seasonal_semi_daily',
       'STL_residual_semi_daily', 'STL_trend_semi_weekly',
       'STL_seasonal_semi_weekly', 'STL_residual_semi_weekly',
       'lagged_demand_1', 'lagged_demand_2', 'lagged_demand_3',
       'lagged_demand_4', 'lagged_demand_5', 'lagged_sum_passenger_count_1',
       'lagged_sum_passenger_count_2', 'lagged_sum_passenger_count_3',
       'lagged_sum_passenger_count_4', 'lagged_sum_passenger_count_5',
       'lagged_sum_trip_durations_1', 'lagged_sum_trip_durations_2',
       'lagged_sum_trip_durations_3', 'lagged_sum_trip_durations_4',
       'lagged_sum_trip_durations_5', 'lagged_Bronx_1', 'lagged_Bronx_2',
       'lagged_Bronx_3', 'lagged_Bronx_4', 'lagged_Bronx_5',
       'lagged_Manhattan_1', 'lagged_Ma

Improved Model II

Xgboost is a better solution to address high-dimensional features and multicollinearity issues since (i) it has embedded feature selection process; and (ii) the tree based algorithms are not directly impacted by multicollinearity. Therefore, we implemented the xgboost algorithm on the training data, and optimize the model by tune the parameter values to achieve the best performance on the test-set. As a result, we received the optimal MAPE 0.067.

We also explored the feature importance of the xgboost model, and find the nonzero important features are: 

  * the last hour demand
  * STL lagged predictions 
  * the last hour total trip durations
  * HW weekly seasonaly predictions
  * the last hour total passengers
  * the total drop-offs at Manhattan 3 hours before


The last two features are interesting since they may be viewed as evidence for our hypothesis: that the Bronx taxi demand is subject to impact of gatherings and events in NYC, especially at Manhattan.












In [480]:
import numpy as np
import xgboost as xgb

# Define the hyperparameter grid
depth_ = np.asarray([2, 3, 5])
estimators_list = np.asarray([300,  500, 800])
colsample_bytree = np.asarray([0.3,  0.5,  0.65])
subsample = np.asarray([0.7, 0.85, 0.95])

# Initialize variables to store the best model and error
best_params = None
lowest_error = float('inf')

# Loop over all combinations of hyperparameters
for depth in depth_:
    for estimator in estimators_list:
        for colsample in colsample_bytree:
            for subsample_value in subsample:
                # Initialize the model with the current hyperparameters
                x_model = xgb.XGBRegressor(
                    max_depth=depth,
                    n_estimators=estimator,
                    colsample_bytree=colsample,
                    subsample=subsample_value,
                    random_state=42  # Ensure reproducibility
                )
                
                # Train the model
                x_model.fit(X_train, y_train)

                # Make predictions on the test set
                y_pred_test = x_model.predict(X_test)

                # Calculate MAPE on the test set
                error = mean_absolute_percentage_error(y_test, y_pred_test)
                
                # Check if the current combination has the lowest error
                if error < lowest_error:
                    lowest_error = error
                    best_params = {
                        'max_depth': depth,
                        'n_estimators': estimator,
                        'colsample_bytree': colsample,
                        'subsample': subsample_value
                    }

                # Output progress and current error
                print(f"Depth: {depth}, Estimators: {estimator}, ColSample: {colsample}, Subsample: {subsample_value} => MAPE: {error:.4f}")

# Output the best hyperparameters
print("Best Parameters:")
print(best_params)
print(f"Lowest MAPE on test set: {lowest_error:.4f}")


Depth: 2, Estimators: 300, ColSample: 0.3, Subsample: 0.7 => MAPE: 0.1428
Depth: 2, Estimators: 300, ColSample: 0.3, Subsample: 0.85 => MAPE: 0.1372
Depth: 2, Estimators: 300, ColSample: 0.3, Subsample: 0.95 => MAPE: 0.1372
Depth: 2, Estimators: 300, ColSample: 0.5, Subsample: 0.7 => MAPE: 0.0864
Depth: 2, Estimators: 300, ColSample: 0.5, Subsample: 0.85 => MAPE: 0.0910
Depth: 2, Estimators: 300, ColSample: 0.5, Subsample: 0.95 => MAPE: 0.1090
Depth: 2, Estimators: 300, ColSample: 0.65, Subsample: 0.7 => MAPE: 0.1381
Depth: 2, Estimators: 300, ColSample: 0.65, Subsample: 0.85 => MAPE: 0.1330
Depth: 2, Estimators: 300, ColSample: 0.65, Subsample: 0.95 => MAPE: 0.1321
Depth: 2, Estimators: 500, ColSample: 0.3, Subsample: 0.7 => MAPE: 0.1136
Depth: 2, Estimators: 500, ColSample: 0.3, Subsample: 0.85 => MAPE: 0.1107
Depth: 2, Estimators: 500, ColSample: 0.3, Subsample: 0.95 => MAPE: 0.1123
Depth: 2, Estimators: 500, ColSample: 0.5, Subsample: 0.7 => MAPE: 0.0748
Depth: 2, Estimators: 500, 

In [494]:
MAPE_IM3 = lowest_error

#save best_params in csv file
best_params_df = pd.DataFrame(best_params, index=[0])
best_params_df.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/best_params.csv', index=False)

#draw the importance plot of the xgboost model
# Initialize the XGBoost model with the best hyperparameters
x_model_best = xgb.XGBRegressor(
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    colsample_bytree=best_params['colsample_bytree'],
    subsample=best_params['subsample'],
    random_state=42  # Ensure reproducibility
)

# Fit the model on the training data
x_model_best.fit(X_train, y_train)

# Get the feature importances
importances_xgb = x_model_best.feature_importances_

# Sort the feature importances in descending order
indices_xgb = np.argsort(importances_xgb)[::-1]

# show the top 20 features
indices_xgb = indices_xgb[:20]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importance of XGBoost Model")
plt.barh(range(len(indices_xgb)), importances_xgb[indices_xgb], align="center")
plt.yticks(range(len(indices_xgb)), X_train.columns[indices_xgb])
plt.xlabel("Relative Importance")
plt.gca().invert_yaxis()  # Invert y-axis to have the highest importance on top
plt.tight_layout()

# Save the plot as an image
plt.savefig("imp_model3.png")
plt.close()

#generate a table with 4 rows, 3 columns.
# 1st column: the names of the models: WMA, RF_HW, RF_no_HW, XGBoost
# 2nd column: the values of  wma2, mape_IM1, mape_IM2, MAPE_IM3
# thrid columns: Notes: weighted moving average with window size 2, Random Forest with all features, Random Forest without Holt-Winters, XGBoost with all features

# Create a DataFrame with the model names, MAPE values, and notes
model_comparison = pd.DataFrame({
    'Model': ['WMA', 'RF_HW', 'RF_no_HW', 'XGBoost'],
    'MAPE': [wma2, mape_IM1, mape_IM2, MAPE_IM3],
    'Notes': [
        'Weighted Moving Average with window size 2',
        'Random Forest with all features',
        'Random Forest without Holt-Winters',
        'XGBoost with all features'
    ]
})
#save dataframe to a csv file
model_comparison.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/model_comparison.csv', index=False)

#print table
print(model_comparison)

      Model      MAPE                                       Notes
0       WMA  0.181333  Weighted Moving Average with window size 2
1     RF_HW  0.193581             Random Forest with all features
2  RF_no_HW  0.150511          Random Forest without Holt-Winters
3   XGBoost  0.066465                   XGBoost with all features


### Forecasting the First Week of September 2024


   - Using your best model, forecast the hourly demand for taxis in the Bronx for the first week of September 2024. Explain any additional steps or assumptions you took into consideration for this forecasting task, such as seasonal trends, external factors, or potential anomalies. 

We use our best model XGboost to make predictions on the first week of 2024. In order to make prediction on September 2024, we need to generate hourly feature values for these XGboost predictions with non-zero importance. However, we do not have 2024 Bronx taxi demand data to deduct the required predictors. Even though we can manually curate some features such as temperature and precipitation, day-of-the-week ,...etc, however, these features are of zero importance for XGboost predictions. In fact, the xgboost model are major driven by the seasonal and trend features of demand. 

Therefore, we will use the predicted taxi demand of frist week of September 2023 to forecast the First Week of September 2024. The assumption is that the multiple seasonality and trend is the major factor of taxi demands, and this assumption is supported by our best model.




In [502]:
#make prediction of the first week of September 2023 using the best model
# Load the data for the first week of September 2023

#first take the last 720 rows of the X_train and y_train
X_sep = X_train[-720:]
y_sep = y_train[-720:]
#then take the first 168 rows of the X_train and y_train
X_sep_week1 = X_sep[:168]
y_sep_week1 = y_sep[:168]

#make prediction on X_sep_week1 and round the prediction
y_pred_sep_week1 = x_model_best.predict(X_sep_week1).round(0)

#save predictions
y_pred_sep_week1 = pd.DataFrame(y_pred_sep_week1, columns=['num_demands'])
y_pred_sep_week1.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/Forcast_sep_week1.csv', index=False)

#print length of y_pred_sep_week1
print('Length of y_pred_sep_week1: ', len(y_pred_sep_week1))

Length of y_pred_sep_week1:  168


### Optimal Number of Taxis


In order to determine the optimal number of taxis required to meet the predicted hourly demand in the Bronx, I will make the following assumptions:

  * the houly average trip durations in the first week, September 2023 is a reliable indicator of that in first week, September 2024.

  * The taxi are constantly avaiable during the hour between trips. i.e. the Taxi drivers are constantly on duty when deployed. The pick-up time for taxi driver to get back to Bronx for the next request can be ignored.

  * Oversupply: If the taxi supply is over the true demand, then the idle driver will lead to operational cost. The minimum wage at NYC is $15 per hour, while a recent NYC law have required companies to pay delivery workers $17.96 an hour. In our calculation, we will use a=$17.96 hourly pay as the baseline idle cost.i.e. Every hour every idle taxi driver will generate extra $17.96 operational cost.

  * undersupply: If the taxi supply is under the true demand, then the cost is (i) revenue oppertunity cost, which we will assume to be $19.62 per-trip according to a NYC article (https://www.nytimes.com/2022/11/17/nyregion/taxi-fare-hike-nyc.html); plus (ii) customer wait times will increase, leading to dissatisfaction. it will bring future potential revenue lost if the customers seek service from a competitor or public transportaion. Given this, I would make assumption that a high percentage of customers, specifically 80% would drop-off the demand pool. i.e. if the surplus of taxi demands are not met in this hour, then only 0.2 of the surplus demands last hour will be added to the demand pool of the next hour.


We conduct simulation analysis by simulating different values of taxi supplies: S. i.e. the number of Taxis deployed. For each value of S ranging from 10 to 100, we will calculate the expected total revenue for the first week, September, 2024. Specifically, the calculation goes as follows:

  * For the first hour, let D=the number of demands, TD = the avearge trip durations (minutes) of this hour, S = supply of taxi.
  * If S * 60 >= D * TD, then the revenue of this hour is D * 19.62 - (S * 60 - D * TD ) * ( 17.96/60 ), i.e. income - idle driver cost.
  * If S * 60 < D * TD, then the revenue of this hour is 19.62 * ( S * 60 / TD ).  Update the next hour demand by D = D + 0.2 *(D - S * 60 /TD). Note that we recognize this approach may result in fractional demand for the next hour. However, we expect that, by the law of large numbers, the approximate distribution will converge to the true demand distribution over time.


Conclusion: 

The optimal number of taxi supply is 19 for first week of September, 2024. Correspondingly, the expected total revenue is $25057.32 before expenditures and deductions. The figure below shows the relationship between the total revenue and supply of taxi.




In [510]:
#take the subset of stl_fourier_features that month =9 and day <=7
sep_week1 = stl_fourier_features[(stl_fourier_features['month'] == 9) & (stl_fourier_features['week_of_month'] == 1)]
#print head
#print(sep_week1.head())

#select only these features: sum_trip_durations  avg_trip_durations in the sep_week1, drop index
sep_week1 = sep_week1[['sum_trip_durations', 'avg_trip_durations']].reset_index(drop=True)

#print dimension
print('Dimension of sep_week1: ', sep_week1.shape)

#combine the sep_week1 with y_pred_sep_week1 by column
sep_week1 = pd.concat([sep_week1, y_pred_sep_week1], axis=1)

#save the sep_week1
sep_week1.to_csv('/Users/laijiang/Documents/Pers/datatest/Data/save/Forcast_sep_week1.csv', index=False)



# Constants
fare_per_trip = 19.62  # Revenue per trip
idle_cost_per_minute = 17.96 / 60  # Idle cost per minute per taxi
supply_range = range(10, 30)  # Taxi supply range for simulation
simulated_week_revenue = []


# Simulate the revenue for each supply level (S) for the entire week
for supply in supply_range:
    total_revenue = 0  # Accumulate revenue over the week
    data_simulation = sep_week1.copy()  # Create a copy of the data for simulation
    
    for i in range(len(data_simulation)):
        D = data_simulation.loc[i, 'num_demands']  # Number of demands in this hour
        TD = data_simulation.loc[i, 'avg_trip_durations']  # Average trip duration in minutes
        S = supply  # Supply of taxis (S)
        
        # Case 1: Supply is enough to meet demand
        if S * 60 >= D * TD:
            # Calculate revenue for the hour: income - idle driver cost
            revenue = D * fare_per_trip - (S * 60 - D * TD) * idle_cost_per_minute /60

        else:
            # Case 2: Supply is not enough to meet demand
            revenue = fare_per_trip * (S * 60 / TD)
            # Update demand for the next hour
            if i + 1 < len(data_simulation):
                next_demand = data_simulation.loc[i + 1, 'num_demands']
                

                additional_demand = 0.2 * (D - S * 60 / TD)  # Fraction of unmet demand added to next hour
                

                
                # Explicitly cast the additional demand to float32 to avoid warning
                data_simulation.loc[i + 1, 'num_demands'] = np.float32(next_demand + additional_demand)
        
        total_revenue += revenue  # Accumulate total revenue for the week
    
    # Append total revenue for the current supply level
    simulated_week_revenue.append((S, total_revenue))

# Convert the results to a DataFrame and sort by revenue
simulated_revenue_df = pd.DataFrame(simulated_week_revenue, columns=['Taxi_Supply', 'Total_Revenue'])
optimal_supply = simulated_revenue_df.sort_values(by='Total_Revenue', ascending=False).iloc[0]

# Display the results
print("Optimal Supply of Taxis:", optimal_supply['Taxi_Supply'])
print("Total Revenue with Optimal Supply:", optimal_supply['Total_Revenue'])


plt.plot(simulated_revenue_df['Taxi_Supply'], simulated_revenue_df['Total_Revenue'])
plt.xlabel('Taxi Supply')
plt.ylabel('Total Revenue')
plt.title('Taxi Supply vs Total Revenue')
plt.grid(True)
#save figure
plt.savefig("revenue_supply.png")
plt.close()


Dimension of sep_week1:  (168, 2)
Optimal Supply of Taxis: 19.0
Total Revenue with Optimal Supply: 25057.31590648646
